In [4]:
import paramiko

ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh_client.connect(hostname="182.92.85.148", username="omm", password="opgs1234321#")

In [7]:
stdin, stdout, stderr = ssh_client.exec_command('whoami')
output = stdout.read().decode()
print(type(output),output)
if output=="omm":
    print("hello")

<class 'str'> omm



In [10]:
import re
str1= output
pattern = r"[a-zA-Z]+\n?"
str2 = re.sub(pattern, lambda m: m.group().strip(), str1)
print(str2)
if str2=="omm":
    print("hello")

omm
hello


In [5]:
stdin, stdout, stderr = ssh_client.exec_command('gs_dump -U jikun -f /home/tianjikun/schema/test.sql -p 15400 dwg -n public -F p')
stdin.write('tianjikun123@\n')
stdin.flush()
output = stdout.read().decode()
print(output)

gs_dump[port='15400'][dwg][2023-07-12 17:17:00]: The total objects number is 520.
gs_dump[port='15400'][dwg][2023-07-12 17:17:00]: [100.00%] 520 objects have been dumped.
gs_dump[port='15400'][dwg][2023-07-12 17:17:01]: dump database dwg successfully
gs_dump[port='15400'][dwg][2023-07-12 17:17:01]: total time: 3109  ms



In [6]:
# sftp_client = ssh_client.open_sftp()
# with sftp_client.open("/home/tianjikun/schema/test.sql", 'r') as f:
#     content=f.read()
sftp_client = ssh_client.open_sftp()
sftp_client.get(localpath='data/test.sql', remotepath='/home/tianjikun/schema/test.sql')
sftp_client.close()

In [21]:
with open("data/test.sql", 'r') as f:
    content=f.read()
print(type(content),len(content))

<class 'str'> 2931820


In [49]:
import re
content_split=re.split('[\n]*;+[\n]*',content)
print(len(content_split))
# content_split = re.sub('\n', '', content_split)
real_content=[]
for it in content_split:
    if "CREATE" in it and "GRANT" not in it:
        print(it)
        real_content.append(it)

567
 Tablespace: 
--

CREATE TABLE tb01 (
    col1 double precision,
    col2 integer,
    col3 integer,
    col4 double precision
)
WITH (orientation=row, compression=no)
 Tablespace: 
--

CREATE TABLE tb02 (
    col1 double precision,
    col2 integer,
    col3 integer,
    col4 double precision
)
WITH (orientation=row, compression=no)
 Tablespace: 
--

CREATE TABLE tb03 (
    col1 double precision,
    col2 integer,
    col3 integer,
    col4 double precision
)
WITH (orientation=row, compression=no)
 Tablespace: 
--

CREATE TABLE tb04 (
    col1 double precision,
    col2 integer,
    col3 integer,
    col4 double precision
)
WITH (orientation=row, compression=no)
 Tablespace: 
--

CREATE TABLE tb05 (
    col1 double precision,
    col2 integer,
    col3 integer,
    col4 double precision
)
WITH (orientation=row, compression=no)
 Tablespace: 
--

CREATE TABLE tb06 (
    col1 double precision,
    col2 integer,
    col3 integer,
    col4 double precision
)
WITH (orientation=row, comp

In [50]:
import re

# create_table_sql=real_content
create_table_sql=''' Tablespace: 
--

CREATE TABLE tb23 (
    col1 double precision,
    col2 integer,
    col3 integer,
    col4 double precision
)
WITH (orientation=row, compression=no)
'''

create_table_sql = "CREATE TABLE example_table (id int, name varchar(255), age int)"
table_name_pattern = r"CREATE TABLE (\w+)"
column_pattern = r"(\w+) ([a-zA-Z]+)(\(\d+\))?"

table_name_match = re.search(table_name_pattern, create_table_sql)
table_name = table_name_match.group(1)

columns = []
column_matches = re.findall(column_pattern, create_table_sql)
for column_match in column_matches:
    column_name = column_match[0]
    column_type = column_match[1]
    if column_name.upper()=="CREATE":
        pass
    else:
        columns.append((column_name, column_type))

print(f"Table name: {table_name}")
print("Columns:")
for column in columns:
    print(f"- Name: {column[0]}, Type: {column[1]}")

Table name: example_table
Columns:
- Name: id, Type: int
- Name: name, Type: varchar
- Name: age, Type: int


In [51]:
def parseSQL2json(create_table_sql):
    table_name_pattern = r"CREATE TABLE (\w+)"
    column_pattern = r"(\w+) ([a-zA-Z]+)(\(\d+\))?"

    table_name_match = re.search(table_name_pattern, create_table_sql)
    
    if table_name_match==None:
        print("fatal error : grammar incorrect.")
        print("error sql : ",create_table_sql)
        
    table_name = table_name_match.group(1)
    
    columns = []
    column_matches = re.findall(column_pattern, create_table_sql)
    for column_match in column_matches:
        column_name = column_match[0]
        column_type = column_match[1]
        if column_name.upper()=="CREATE":
            pass
        else:
            columns.append((column_name, column_type))

    print(f"Table name: {table_name}")
    print("Columns:")
    for column in columns:
        print(f"- Name: {column[0]}, Type: {column[1]}")

In [46]:
parseSQL2json("CREATE TABLE example_table (id int, name varchar(255), age int)")

Table name: example_table
Columns:
- Name: id, Type: int
- Name: name, Type: varchar
- Name: age, Type: int


In [47]:
print(real_content[0])
print(parseSQL2json(real_content[0]))

 Tablespace: 
--

CREATE TABLE tb01 (
    col1 double precision,
    col2 integer,
    col3 integer,
    col4 double precision
)
WITH (orientation=row, compression=no)
Table name: tb01
Columns:
- Name: col1, Type: double
- Name: col2, Type: integer
- Name: col3, Type: integer
- Name: col4, Type: double
None


In [52]:
for it in real_content:
    parseSQL2json(it)

Table name: tb01
Columns:
- Name: col1, Type: double
- Name: col2, Type: integer
- Name: col3, Type: integer
- Name: col4, Type: double
Table name: tb02
Columns:
- Name: col1, Type: double
- Name: col2, Type: integer
- Name: col3, Type: integer
- Name: col4, Type: double
Table name: tb03
Columns:
- Name: col1, Type: double
- Name: col2, Type: integer
- Name: col3, Type: integer
- Name: col4, Type: double
Table name: tb04
Columns:
- Name: col1, Type: double
- Name: col2, Type: integer
- Name: col3, Type: integer
- Name: col4, Type: double
Table name: tb05
Columns:
- Name: col1, Type: double
- Name: col2, Type: integer
- Name: col3, Type: integer
- Name: col4, Type: double
Table name: tb06
Columns:
- Name: col1, Type: double
- Name: col2, Type: integer
- Name: col3, Type: integer
- Name: col4, Type: double
Table name: tb07
Columns:
- Name: col1, Type: double
- Name: col2, Type: integer
- Name: col3, Type: integer
- Name: col4, Type: double
Table name: tb08
Columns:
- Name: col1, Type: do

In [55]:
import json
json_data={
    "a":1,
    "b":{
        "x":[1,2,3]
    }
}
with open("data/res.json", "w") as f:
    json.dump(json_data, f, indent=4)
print(json_data["a"])

1


In [1]:
def parseSQL2json2(create_table_sql):
    one_table={
        "Table Name": "",
        "Table Columns": [],
        "Column Distribution" : [],
        "Primary Key": {
            "Name": "",
            "Data Type": ""
        },
        "Foreign Key": [
            {
                "Foreign Key Name": "",
                "Foreign Key Type": "",
                "Referenced Table": "",
                "Referenced Primary Key": "",
                "Referenced Primary Key Type": ""
            }
        ]
    }
    table_name_pattern = r"CREATE TABLE (\w+)"
    column_pattern = r"(\w+) ([a-zA-Z]+)(\(\d+\))?"
    table_name_match = re.search(table_name_pattern, create_table_sql)
    
    if table_name_match==None:
        print("fatal error : grammar incorrect.")
        print("error sql : ",create_table_sql)
    
    table_name = table_name_match.group(1)
    
    columns = []
    column_matches = re.findall(column_pattern, create_table_sql)
    
    for column_match in column_matches:
        column_name = column_match[0]
        column_type = column_match[1]
        if column_name.upper()=="CREATE":
            pass
        else:
            # columns.append((column_name, column_type))
            if column_type=="varchar":
                one_column={
                    "Column Name": column_name,
                    "Data Type": column_type,
                    "Data Distribution": ["minv","maxv"]
                }
            else:  
                one_column={
                    "Column Name": column_name,
                    "Data Type": column_type,
                    "Data Distribution": [0,100]
                }
            columns.append(one_column)
            
    one_table["Table Name"]=table_name
    one_table["Table Columns"]=columns
    one_table["Column Distribution"]=[1.0/len(columns) for i in range(len(columns))]

    return one_table
    

In [2]:
json_data={
    "Table Schema": "noobschema",
    "Tables": [],
    "Constraints": {
      "size of workload" : 0,
      "read write ratio": 1,
      "average table num": [],
      "table-query access distribution": [],
      "query comparison operator ratio" : [],
      "table domain distribution" : [],
      "query logic predicate num" : [],
      "average aggregation operator num" : [],
      "average query colomn num":[],
      "group by ratio if read SQL":[],
      "order by desc or asc if grouped":[]
    },
    "Generation File":""
}

In [3]:
tables=[]
for it in real_content:
    one_table=parseSQL2json2(it)
    tables.append(one_table)
json_data["Tables"]=tables
with open("data/res.json", "w") as f:
    json.dump(json_data, f, indent=4)

NameError: name 'real_content' is not defined

In [ ]:
ssh_client.close()

In [1]:
import re

pattern = r'((\d{2})-(\d{2})-(\d{4}))'
text = 'Date: 23-08-2023'

match = re.search(pattern, text)
if match:
    # 返回整个匹配的字符串
    print('Match:', match.group())  # 输出: Match: 23-08-2023

    # 嵌套调用 group() 函数获取每个匹配组的内容
    print('Group 1:', match.group(1))  # 输出: Group 1: 23-08-2023
    print('Group 2:', match.group(2))  # 输出: Group 2: 23
    print('Group 3:', match.group(3))  # 输出: Group 3: 08
    print('Group 4:', match.group(4))  # 输出: Group 4: 2023
else:
    print('No match')

Match: 23-08-2023
Group 1: 23-08-2023
Group 2: 23
Group 3: 08
Group 4: 2023


In [4]:
import re
import json
import argparse

def parseSQL2json2(create_table_sql):
    one_table={
        "Table Name": "",
        "Table Columns": [],
        "Column Distribution" : [],
        "Primary Key": {
            "Name": "",
            "Data Type": ""
        },
        "Foreign Key": [
            {
                "Foreign Key Name": "",
                "Foreign Key Type": "",
                "Referenced Table": "",
                "Referenced Primary Key": "",
                "Referenced Primary Key Type": ""
            }
        ]
    }
    table_name_pattern = r"CREATE TABLE (\w+)"
    column_pattern = r"(\w+)\s+([a-zA-Z]+)(\(\d+\))?"
    table_name_match = re.search(table_name_pattern, create_table_sql)
    
    if table_name_match==None:
        print("fatal error : grammar incorrect.")
        print("error sql : ",create_table_sql)
    
    table_name = table_name_match.group(1)
    
    columns = []
    column_matches = re.findall(column_pattern, create_table_sql)
    
    for column_match in column_matches:
        column_name = column_match[0]
        column_type = column_match[1]
        if column_name.upper()=="CREATE":
            pass
        else:
            # columns.append((column_name, column_type))
            if column_type=="varchar":
                one_column={
                    "Column Name": column_name,
                    "Data Type": column_type,
                    "Data Distribution": ["minv","maxv"]
                }
            else:  
                one_column={
                    "Column Name": column_name,
                    "Data Type": column_type,
                    "Data Distribution": [0,100]
                }
            columns.append(one_column)
            
    one_table["Table Name"]=table_name
    one_table["Table Columns"]=columns
    one_table["Column Distribution"]=[1.0/len(columns) for i in range(len(columns))]

    return one_table

In [7]:
with open("data/test.sql", 'r') as f:
    content=f.read()
print(type(content),len(content))

content_split=re.split('[\n]*;+[\n]*',content)
print(len(content_split))
# content_split = re.sub('\n', '', content_split)
real_content=[]

for it in content_split:
    # if "CREATE" in it and "GRANT" not in it and "INDEX" not in it:
        # print(it)
        # real_content.append(it)
    filter_pattern = r"(CREATE TABLE .*)WITH"
    if re.search(pattern=filter_pattern,string=it,flags=re.DOTALL)!=None:
        # print(it)
        real_content.append(it)

tables=[]
for it in real_content:
    one_table=parseSQL2json2(it)
    tables.append(one_table)
    print(one_table["Table Name"])
    


<class 'str'> 97146
2337
sbtest1
sbtest10
sbtest11
sbtest12
sbtest13
sbtest14
sbtest15
sbtest16
sbtest17
sbtest18
sbtest19
sbtest2
sbtest20
sbtest21
sbtest22
sbtest23
sbtest24
sbtest25
sbtest26
sbtest27
sbtest28
sbtest29
sbtest3
sbtest30
sbtest31
sbtest32
sbtest33
sbtest34
sbtest35
sbtest36
sbtest37
sbtest38
sbtest39
sbtest4
sbtest40
sbtest41
sbtest42
sbtest43
sbtest44
sbtest45
sbtest46
sbtest47
sbtest48
sbtest49
sbtest5
sbtest50
sbtest51
sbtest52
sbtest53
sbtest54
sbtest55
sbtest56
sbtest57
sbtest58
sbtest59
sbtest6
sbtest60
sbtest61
sbtest62
sbtest63
sbtest64
sbtest65
sbtest66
sbtest67
sbtest68
sbtest69
sbtest7
sbtest70
sbtest71
sbtest72
sbtest73
sbtest74
sbtest75
sbtest76
sbtest77
sbtest78
sbtest79
sbtest8
sbtest80
sbtest9


In [6]:
from pglast import parse_sql

def parse_create_table(sql):
    result = parse_sql(sql)
    for stmt in result.tree:
        if stmt['stmt'] == 'CreateStmt':
            table_name = stmt['relation']['relname']
            print('Table Name:', table_name)

            columns = stmt['tableElts']
            for column in columns:
                column_name = column['colname']
                data_type = column['typeName']['names'][0]
                print('Column Name:', column_name)
                print('Data Type:', data_type)

# 测试示例
sql_statement = '''
    CREATE TABLE "createsjavaslobstable" (
        name character varying (700),
        lob blob,
        loadtime timestamp(0) without time zone
    )WITH (orientation=row, compression=no,storage_type=astore)
    DISTRIBUTE BY HASH (name);
'''

parse_create_table(sql_statement)

ParseError: syntax error at or near "row", at index 171

In [8]:
sql_statement = '''
CREATE TABLE sbtest1 (
    id integer NOT NULL,
    k integer DEFAULT 0 NOT NULL,
    c character(120) DEFAULT NULL::bpchar NOT NULL,
    pad character(60) DEFAULT NULL::bpchar NOT NULL
)
WITH (orientation=row, compression=no);

'''
import sqlparse
parsed=sqlparse.parse(sql_statement)
for statement in parsed:
    for token in statement.tokens:
        print(token.ttype,token)

Token.Text.Whitespace.Newline 

Token.Keyword.DDL CREATE
Token.Text.Whitespace  
Token.Keyword TABLE
Token.Text.Whitespace  
None sbtest1
Token.Text.Whitespace  
None (
    id integer NOT NULL,
    k integer DEFAULT 0 NOT NULL,
    c character(120) DEFAULT NULL::bpchar NOT NULL,
    pad character(60) DEFAULT NULL::bpchar NOT NULL
)
Token.Text.Whitespace.Newline 

Token.Keyword.CTE WITH
Token.Text.Whitespace  
None (orientation=row, compression=no)
Token.Punctuation ;


In [134]:
import sqlparse

def extract_columns_and_datatypes(sql):
    parsed = sqlparse.parse(sql)
    
    columns = []
    datatypes = []
    
    for statement in parsed:
        if statement.get_type() == 'SELECT':
            # 处理 SELECT 语句
            for token in statement.tokens:
                if token.is_group() and token.get_type() == 'Parenthesis':
                    # 获取列和数据类型
                    for sub_token in token.tokens:
                        if sub_token.get_type() == 'Identifier':
                            columns.append(sub_token.get_real_name())
                        elif sub_token.get_type() == 'DataType':
                            datatypes.append(sub_token.get_real_name())
                            
        elif statement.get_type() == 'CREATE':
            # 处理 CREATE TABLE 语句
            for token in statement.tokens:
                if token.get_type() == 'Keyword' and token.value.upper() == 'TABLE':
                    # 获取表名
                    table_name = statement.tokens[statement.tokens.index(token) + 2].get_real_name()
                elif token.get_type() == 'Keyword' and token.value.upper() == 'COLUMN':
                    # 获取列和数据类型
                    column_token = statement.tokens[statement.tokens.index(token) + 1]
                    if column_token.get_type() == 'Identifier':
                        columns.append(column_token.get_real_name())
                    elif column_token.get_type() == 'Parenthesis':
                        for sub_token in column_token.tokens:
                            if sub_token.get_type() == 'Identifier':
                                columns.append(sub_token.get_real_name())
                            elif sub_token.get_type() == 'DataType':
                                datatypes.append(sub_token.get_real_name())
                            
    return table_name, columns, datatypes

# 示例 SQL 语句
sql = "CREATE TABLE my_table (id INT, name VARCHAR(50), age INT);"

table_name, columns, datatypes = extract_columns_and_datatypes(sql)

print("Table Name:", table_name)
print("Columns:", columns)
print("Datatypes:", datatypes)

AttributeError: 'Token' object has no attribute 'get_type'

In [22]:
import psqlparse

# 测试示例
sql_statement = '''
Tablespace:
--
CREATE TABLE sbtest1 (
    id integer NOT NULL,
    k integer DEFAULT 0 NOT NULL,
    c character(120) DEFAULT NULL::bpchar NOT NULL,
    pad character(60) DEFAULT NULL::bpchar NOT NULL
)
WITH (orientation=row, compression=no);

'''
import re
pa="(CREATE TABLE .*)WITH"
ans=re.search(pattern=pa,string=sql_statement,flags=re.DOTALL).group(1)
print(ans,"\n")
# sql_statement="SELECT * from mytabl"
statements=psqlparse.parse(ans)
index_=1
print(statements[0]['CreateStmt']['relation']['RangeVar']['relname'])
for i in statements[0]['CreateStmt']['tableElts']:
    # 各个列
    for j,k in i.items():
        # print(j,":")
        print(f"{index_} : ")
        for m,n in k.items():
            # print(n)
            if m=="typeName":
                index_+=1
                
                print("\tcoltype : ",n['TypeName']['names'][len(n['TypeName']['names'])-1]['String']['str'])
                # if n['TypeName']['names'][len(n['TypeName']['names'])-1]['String']['str']=="varchar":
                #     print("\ttypemod : ",n['TypeName']['typmods'][0]['A_Const']['val']['Integer']['ival'])
                # if n['TypeName']['names'][len(n['TypeName']['names'])-1]['String']['str']=='blob':
                #     print("\t\t",n['TypeName']['typemod'])
                if n['TypeName']['names'][len(n['TypeName']['names'])-1]['String']['str']=='timestamp':
                    print("\ttypemod : ",n['TypeName']['typmods'][0]['A_Const']['val']['Integer']['ival'])
                elif n['TypeName']['names'][len(n['TypeName']['names'])-1]['String']['str']=='numeric':
                    print("\ttypemod : ",n['TypeName']['typemod'])
                    # print("\ttypemod : ",n['TypeName']['typmods'][0]['A_Const']['val']['Integer']['ival'])
                    # print("\ttypemod : ",n['TypeName']['typmods'][1]['A_Const']['val']['Integer']['ival'])
                    # print("\ttypemod : ",n['TypeName']['typmods'][1]['A_Const'])
                elif n['TypeName']['names'][len(n['TypeName']['names'])-1]['String']['str']=='bpchar':
                    print("\ttypemod : ",n['TypeName']['typmods'][0]['A_Const']['val'])
                else:
                    print("\ttypemod : ",n['TypeName']['typemod'])
            elif m=="colname":
                print("\tcolname : ",n)
            else:
                pass

CREATE TABLE sbtest1 (
    id integer NOT NULL,
    k integer DEFAULT 0 NOT NULL,
    c character(120) DEFAULT NULL::bpchar NOT NULL,
    pad character(60) DEFAULT NULL::bpchar NOT NULL
)
 

sbtest1
1 : 
	colname :  id
	coltype :  int4
	typemod :  -1
2 : 
	colname :  k
	coltype :  int4
	typemod :  -1
3 : 
	colname :  c
	coltype :  bpchar
	typemod :  {'Integer': {'ival': 120}}
4 : 
	colname :  pad
	coltype :  bpchar
	typemod :  {'Integer': {'ival': 60}}


In [71]:
import re
def extract_text_in_parentheses(text):
    pattern = r'\((.*?)\)'  # 匹配括号内的内容，非贪婪模式
    matches = re.findall(pattern, text)
    return matches

# 测试示例
string = "这是一个(括号)示例(字符串)，(提取)其中的部分。"
result = extract_text_in_parentheses(string)
print(result)  # 输出: ['括号', '提取']

AttributeError: 'NoneType' object has no attribute 'group'

In [1]:
import random
from datetime import datetime, timedelta

def generate_random_time():
    # 生成一个随机的日期
    start_date = datetime(2000, 1, 1)
    end_date = datetime.now()
    random_date = start_date + random.random() * (end_date - start_date)
    
    # 生成一个随机的时间
    random_time = random_date.time()
    
    return random_time

# 生成随机时间
random_time = generate_random_time()

# 打印结果
print(random_time)
print(type(random_time))

11:25:29.412700
<class 'datetime.time'>


In [2]:
import random
import datetime

def generate_random_timestamp(precision=0):
    # 获取当前时间
    current_time = datetime.datetime.now()

    # 生成随机的年份、月份、日期、小时、分钟和秒数
    year = random.randint(1970, current_time.year)
    month = random.randint(1, 12)
    day = random.randint(1, 28)  # 假设每个月最大为28天
    hour = random.randint(0, 23)
    minute = random.randint(0, 59)
    second = random.randint(0, 59)

    # 生成随机的微秒数
    microsecond = random.randint(0, 999999)

    # 根据精度截断微秒数
    microsecond = microsecond // (10 ** (6 - precision))

    # 创建时间戳
    timestamp = datetime.datetime(year, month, day, hour, minute, second, microsecond)

    return timestamp

# 测试示例
for precision in range(7):
    random_timestamp = generate_random_timestamp(precision)
    print(f"timestamp({precision}): {random_timestamp}")

timestamp(0): 2012-12-16 05:38:06
timestamp(1): 2000-12-17 16:52:55.000001
timestamp(2): 2021-12-12 09:42:19.000036
timestamp(3): 2023-02-23 08:46:52.000971
timestamp(4): 1984-12-11 07:28:48.007749
timestamp(5): 2019-11-25 04:59:06.027845
timestamp(6): 1978-02-10 12:49:19.834272


In [ ]:
-- 创建表 "a"
CREATE TABLE a (
  col_timestamp_0 TIMESTAMP(0),
  col_timestamp_1 TIMESTAMP(1),
  col_timestamp_2 TIMESTAMP(2),
  col_timestamp_3 TIMESTAMP(3),
  col_timestamp_4 TIMESTAMP(4),
  col_timestamp_5 TIMESTAMP(5),
  col_timestamp_6 TIMESTAMP(6)
);

-- 插入数据
INSERT INTO a
VALUES (
  '2023-08-27 12:34:56',
  '2023-08-27 12:34:56.1',
  '2023-08-27 12:34:56.12',
  '2023-08-27 12:34:56.123',
  '2023-08-27 12:34:56.1234',
  '2023-08-27 12:34:56.12345',
  '2023-08-27 12:34:56.123456'
);

In [1]:
import psqlparse
x="ans"
try:
    statements=psqlparse.parse(x)
except Exception as e:
    print("fatal error : sql syntax incorrect.")
    print("wrong sql : {x}")

fatal error : sql syntax incorrect.
wrong sql : {x}


In [2]:
import datetime
import random
def rand_timestamp_sampling2(timestamp1, timestamp2, precision=0):
    # 获取两个时间戳之间的时间差
    time_diff = (timestamp2 - timestamp1).total_seconds()
    # 生成一个随机的时间差（秒数）
    random_seconds = random.uniform(0, time_diff)
    # 创建一个时间间隔对象，以便添加到timestamp1
    time_interval = datetime.timedelta(seconds=random_seconds)
    # 添加时间间隔以生成随机时间戳
    random_timestamp = timestamp1 + time_interval
    # 根据精度截断微秒数
    microsecond = random_timestamp.microsecond // (10 ** (6 - precision))
    random_timestamp = random_timestamp.replace(microsecond=microsecond)

    return random_timestamp

print(rand_timestamp_sampling2(
    timestamp1=datetime.datetime(2000,10,14,2,30),timestamp2=datetime.datetime(2000,10,15,4,40),precision=2)
)


2000-10-15 00:53:23.000077


In [3]:
import re
print(re.search(pattern=r"select *",string="select dsdad",flags=re.DOTALL))
print(re.search(pattern=r"select *",string="selct dsdad",flags=re.DOTALL))

<re.Match object; span=(0, 7), match='select '>
None


In [2]:
def test(t1):
    if t1=="false":
        x=1
    else:
        x=2
    return x
print(test("false"))

1


In [8]:
import datetime
ss="2021-1-1"
ans=datetime.datetime.strptime(ss,"%Y-%m-%d")
print(ans)
print(ans.second)

2021-01-01 00:00:00
0


In [1]:
def rand_num_sampling(l,r,pdg):
    # 先检查概率分布图和区间是否对应
    if r-l+1 != len(pdg):
        print("l is "+str(l)+" r is "+str(r)+" and pdg is "+str(pdg))
        print("error : pdg not match [l,r]")
        return l-1
    one_=sum(pdg)
    
    # 检查概率分布和是否为1
    if abs(one_-1.0)>1e-5:
        print("error : probability sum "+str(one_)+" do not equal 1")
        return l-1
    pdg_=np.array(pdg)
    pdg_=[round(i*10000000) for i in pdg_]
    maxv=sum(pdg_)
    num=random.randint(0,maxv-1)
    index=0
    for i in range(len(pdg_)):
        num=num-pdg_[i]
        if num<0:
            index=i
            break
    # 返回采样值
    return l+index

In [16]:
import numpy as np
import random
def rand_decimal_sampling(precision):
    if precision==0:
        return 0
    rand_int=rand_num_sampling(0,10**precision-1,np.full(10**precision,1.0/(10**precision)))
    return rand_int/(10**precision)

print(rand_decimal_sampling(1))

0.7
